# Creating Decision Trees using System Entropy and Information Gain

## Import Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('data/train.csv')

In [3]:
data.head(n=10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Feature Selection & Data Cleaning

Select important features and substitute/discard missing features.

### Feature Selection

In [6]:
columns_to_drop = ["PassengerId", "Name", "Cabin", "Embarked", "Ticket"]

In [8]:
data_clean = data.drop(columns_to_drop, axis = 1)

In [9]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 48.9+ KB


In [10]:
data_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


### Convert Features to Numeric Values

In [11]:
# Convert Features to numbers.
from sklearn.preprocessing import LabelEncoder

In [13]:
le = LabelEncoder()
data_clean['Sex'] = le.fit_transform(data_clean['Sex'])

In [14]:
data_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [15]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


### Fill NaN Values (Missing Data)

In [17]:
data_clean = data_clean.fillna(data_clean['Age'].mean())

In [18]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [22]:
data_clean.iloc[1]

Survived     1.0000
Pclass       1.0000
Sex          0.0000
Age         38.0000
SibSp        1.0000
Parch        0.0000
Fare        71.2833
Name: 1, dtype: float64

### Split Data into X and Y

In [23]:
input_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
output_cols = ['Survived']

X = data_clean[input_cols]
Y = data_clean[output_cols]

print(X.shape)
print(Y.shape)

(891, 6)
(891, 1)


## Building a Decision Tree using Entropy and Information Gain Information

Entropy = $- /sum (p(x) * log(p(x))$
Information Gain = $ H(S) - /sum(p(x)*H(S)) $

In [55]:
def entropy(col):
    
    elements, counts = np.unique(col, return_counts=True)
    total_elements = float(col.shape[0])
    entropy = 0.0
    
    for count in counts:
        probability = count / total_elements
        entropy += (-1 * probability * np.log2(probability))
        
    return entropy

In [56]:
def divide_data(data_matrix, split_column, threshold):
    
    x_right = pd.DataFrame([], columns=data_matrix.columns)
    x_left = pd.DataFrame([], columns=data_matrix.columns)
    
    for idx in range(data_matrix.shape[0]):
        value = data_matrix[split_column].loc[idx]
        if (value<threshold):
            x_left = x_left.append(data_matrix.loc[idx])
        else:
            x_right = x_right.append(data_matrix.loc[idx])
            
    return x_left, x_right

In [57]:
def information_gain(data_matrix, split_column, threshold):
    
    data_left, data_right = divide_data(data_matrix, split_column, threshold)
    
    # % of total samples that are on left and right
    l = float(data_left.shape[0])/data_matrix.shape[0]
    r = float(data_right.shape[0])/data_matrix.shape[0]
    
    # If ALL examples come to one side!
    if data_left.shape[0] == 0 or data_right.shape[0] == 0:
        return -10000000 # Min Information Gain
    
    info_gain = entropy(data_matrix.Survived) - l*entropy(data_left.Survived) - r*entropy(data_right.Survived)
    
    return info_gain

In [58]:
for column in X.columns:
    print(column)
    print(information_gain(data_clean, column, data_clean[column].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606143
Age
0.001158644038169343
SibSp
0.009584541813400127
Parch
0.015380754493137666
Fare
0.04214069283899541


Maximum Information Game is with column 'Sex' so this is the first node of the decision tree since it offered maximum reduction of entropy/maximum information gain.

## Implementing Decision Tree

In [93]:
class DecisionTree:
    
    def __init__(self, depth=0, max_depth=5):
        self.left = None
        self.right = None
        self.split_column = None
        self.threshold = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None # Prediction at this node.
    
    def train(self, X_train):
        features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
        info_gain = []
        
        # Compute Info Gain for all columns.
        for feature in features:
            info_gain.append(information_gain(X_train, feature, X_train[feature].mean()))
        
        # Select the column with max info gain.
        self.split_column = max_info_gain_col_idx = features[np.argmax(info_gain)]
        self.threshold = X_train[self.split_column].mean()
        
        print("Making tree here with feature = ", self.split_column)
        
        # Split Data
        data_left, data_right = divide_data(X_train, self.split_column, self.threshold)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
        
        # Truly a Leaf Node.
        if data_left.shape[0] == 0 or data_right.shape[0] == 0:
            if X_train['Survived'].mean() >= 0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return
                
        # Early Stopping for depth >= max depth
        if(self.depth >= self.max_depth):
            if X_train['Survived'].mean() >= 0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return
                
        # Recursive Case
        self.left = DecisionTree(depth=self.depth+1)
        self.left.train(data_left)
        self.right = DecisionTree(depth=self.depth+1)
        self.right.train(data_right)
        
        # Set target at every node instead of just leaf nodes.
        if X_train['Survived'].mean() >= 0.5:
            self.target = "Survived"
        else:
            self.target = "Dead"
        
        print("Target = ", self.target)
        return
    
    def predict(self, X_test):
        if X_test[self.split_column] > self.threshold:
            if self.right is None:
                return self.target
            return self.right.predict(X_test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(X_test)

In [80]:
d = DecisionTree()
d.train(data_clean)

Making tree here with feature =  Sex
Making tree here with feature =  Pclass
Making tree here with feature =  Pclass
Making tree here with feature =  Parch
Making tree here with feature =  Age
Making tree here with feature =  Age
Making tree here with feature =  Age
Making tree here with feature =  Parch
Making tree here with feature =  Age
Making tree here with feature =  Fare
Making tree here with feature =  Parch
Making tree here with feature =  Age
Making tree here with feature =  Age
Making tree here with feature =  Age
Making tree here with feature =  Age
Making tree here with feature =  Age
Making tree here with feature =  Age
Making tree here with feature =  Fare
Making tree here with feature =  SibSp
Making tree here with feature =  Fare
Making tree here with feature =  Fare
Making tree here with feature =  Age
Making tree here with feature =  SibSp
Making tree here with feature =  Parch
Making tree here with feature =  Age
Making tree here with feature =  SibSp
Making tree he

## Train-Test Split

Validation Split is used in Decision Trees for finding out the max_depth so as to not overfit.

In [94]:
split = int(0.7*data_clean.shape[0])
train_data = data_clean[:split]
test_data = data_clean[split:]
test_data = test_data.reset_index(drop=True)

In [95]:
print(train_data.shape, test_data.shape)

(623, 7) (268, 7)


In [96]:
dt = DecisionTree()

In [97]:
dt.train(train_data)

Making tree here with feature =  Sex
Making tree here with feature =  Pclass
Making tree here with feature =  Age
Making tree here with feature =  SibSp
Making tree here with feature =  Pclass
Making tree here with feature =  Age
Making tree here with feature =  Age
Target =  Survived
Making tree here with feature =  SibSp
Making tree here with feature =  Parch
Making tree here with feature =  Pclass
Target =  Survived
Target =  Survived
Making tree here with feature =  SibSp
Making tree here with feature =  Fare
Making tree here with feature =  Parch
Making tree here with feature =  Age
Target =  Survived
Making tree here with feature =  Pclass
Making tree here with feature =  Age
Making tree here with feature =  Age
Target =  Survived
Target =  Survived
Target =  Survived
Making tree here with feature =  Parch
Making tree here with feature =  SibSp
Making tree here with feature =  Fare
Making tree here with feature =  Age
Making tree here with feature =  Age
Target =  Survived
Making

In [98]:
print(dt.split_column)
print(dt.threshold)
print(dt.left.split_column)
print(dt.right.split_column)

Sex
0.6292134831460674
Pclass
Fare


In [105]:
ypreds = []
for idx in range(test_data.shape[0]):
    ypreds.append(dt.predict(test_data.loc[idx]))

In [106]:
ypreds

['Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Survived',
 'Dead',
 'Survived',
 'Survived',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survived',
 'Survived',
 'Survived',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survived',
 'Survived',
 'Survived',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Survived',
 'Dead',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Survived',
 'Dead',
 'Survived',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survived',
 'Survived',
 'Dead',
 'Dead',
 'Survived',
 'Dead',
 'Dead',


In [104]:
lr = LabelEncoder()
ypredsLR = lr.fit_transform(ypreds)

In [108]:
acc = (ypredsLR == test_data['Survived']).mean()

In [109]:
acc

0.8171641791044776